In [1]:
# Update and import packages
!pip install -Uqr requirements.txt

# Basic packages
import importlib
from time import time
from pathlib import Path
from progress.bar import Bar
import json
import re
from bs4 import BeautifulSoup

# Data science/NLP packages
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", None)

# AWS packages
import awswrangler as wr
import sagemaker
from sagemaker.sklearn import SKLearn
import boto3

# Local modules
import model
import train
for m in [model, train]:
    importlib.reload(m)

from model import VectorSimilarity, get_fitted_model
from train import combine_dfs

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
# Sanity checks on VectorSimilarity
X = np.array(
    [[0, 1],
     [1, 0],
     [-1, 0]])
y = np.array(['a', 'b', 'c'])

estimator = VectorSimilarity()
estimator = estimator.fit(X, y)
estimator.predict(np.array([1, 2]).reshape(1, -1))

array([['a', 'b', 'c']], dtype='<U1')

In [5]:
# Basic pipeline setup
basic_corpus = [
    'Bees like to make honey',
    'Bears like to eat honey',
    'Bees don\'t like bears',
    'Humans are walking around the park'
]
basic_labels = ['a', 'b', 'c', 'd']

pipe = get_fitted_model(basic_corpus, basic_labels)
pred, score = pipe.predict_score(basic_corpus)
print(pred)
print(score)

Training model...
Took 0.002666473388671875 seconds
[['a' 'b' 'c' 'd']
 ['b' 'a' 'c' 'd']
 ['c' 'b' 'a' 'd']
 ['d' 'c' 'b' 'a']]
[[1.         0.50443175 0.29772523 0.        ]
 [1.         0.50443175 0.29772523 0.        ]
 [1.         0.29772523 0.29772523 0.        ]
 [1.         0.         0.         0.        ]]


In [6]:
# Train and infer on small Data Wrangler dataset
sess = sagemaker.Session()
bucket = sess.default_bucket()

chunksize = 1000
output_content_type = "parquet"
flow_export_id = f"30-23-06-49-58efbaf1"
flow_export_name = f"flow-{flow_export_id}"
s3_output_prefix = f"export-{flow_export_name}/output"
s3_output_path = f"s3://{bucket}/{s3_output_prefix}"

dfs=[]
if output_content_type.upper() == "CSV":
    dfs = wr.s3.read_csv(s3_output_path, chunksize=chunksize)
elif output_content_type.upper() == "PARQUET":
    dfs = wr.s3.read_parquet(s3_output_path, chunked=chunksize)
else:
    print(f"Unexpected output content type {output_content_type}") 

wrangled_df = next(dfs)

X = wrangled_df['bodyText']
y = wrangled_df['url']
pipe.fit(X, y)

pred, score = pipe.predict(X[13:14])
print(list(y[13:14]))

[['https://github.com/aws-amplify/amplify-adminui/issues/12'
  'https://github.com/aws-amplify/amplify-adminui/issues/21'
  'https://github.com/aws-amplify/amplify-adminui/issues/82'
  'https://github.com/aws-amplify/amplify-adminui/issues/28'
  'https://github.com/aws-amplify/amplify-adminui/issues/41'
  'https://github.com/aws-amplify/amplify-adminui/issues/76'
  'https://github.com/aws-amplify/amplify-adminui/issues/78'
  'https://github.com/aws-amplify/amplify-adminui/issues/67'
  'https://github.com/aws-amplify/amplify-adminui/issues/85'
  'https://github.com/aws-amplify/amplify-adminui/issues/4']]
[[1.         0.37623503 0.26797184 0.25792939 0.23923179 0.2152719
  0.19000821 0.18763175 0.17725247 0.17051992]]
['https://github.com/aws-amplify/amplify-adminui/issues/12']


In [7]:
# File helper functions
def list_data_objs():
    secret_name = "SageMakerS3Access"
    region_name = "us-west-2"
    bucket_name = 'githubmachinelearningstack-rawdatabucket79e6ae92-dvgbsz21ce9v'
    bucket_subfolder = 'data/'
    
    secrets = boto3.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    secrets_response = secrets.get_secret_value(SecretId=secret_name)
    secrets_dict = json.loads(secrets_response['SecretString'])
    (access_key, secret_key), = secrets_dict.items()

    s3 = boto3.client('s3')
    data_objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=bucket_subfolder)['Contents']
    data_obj_names = [f"s3://{bucket_name}/{key['Key']}" for key in data_objects]
    
    return data_obj_names


def download_data(filename, data_obj_names):
    dfs = []
    
    with Bar(
        message='Downloading parquets',
        check_tty=False,
        hide_cursor=False,
        max=len(data_obj_names)
    ) as bar:

        for obj_name in data_obj_names:
            df = wr.s3.read_parquet(obj_name)
            dfs.append(df)
            bar.next()

        bar.finish()
        
    df = combine_dfs(dfs)
    df.to_csv(filename)

    return df

def get_data(filename, force_redownload=False):
    start = time()
    data = Path(filename)
    
    if data.is_file() and not force_redownload:
        print('Deserializing data from', filename, '...')
        df = pd.read_csv(filename)
        
    else:
        data_obj_names = list_data_objs()
        df = download_data(filename, data_obj_names)
        
    print('Took', time() - start, 'seconds')    
    return df

In [8]:
# Data helper functions
def query_df(df, **kwargs):
    query = True
    for key, value in kwargs.items():
        query &= (df[key] == value)
        
    result = df[query]
    return result

def compare_vecs():
    print('unimplemented')

def inspect_corpus(vectorizer, df, url):
    print('unimplemented')
    

In [9]:
# Download and compile parquets
df = get_data('training_data.csv', force_redownload=False)

Deserializing data from training_data.csv ...
Took 3.8856701850891113 seconds


In [10]:
# Preprocess training data
corpus_col = 'bodyCleaned'
label_col = 'url'

begin_text = r'.*Describe the bug'
mid_text = r'### Expected behavior|### Reproduction steps|\r\n*'
end_text = r'### Code Snippet.*'
begin_text_feat = r'.*Describe the feature you\'d like to request'
mid_text_feat = r'### Describe the solution you\'d like|### Describe alternatives you\'ve considered'
end_text_feat = r'### Additional context.*'

cases = [
    begin_text,
    mid_text,
    end_text,
    begin_text_feat,
    mid_text_feat,
    end_text_feat
]
pat_cases = '(' + '|'.join(cases) + ')'
pat = re.compile(pat_cases, flags=(re.DOTALL | re.M))

print('Preprocessing data...')
start = time()

# js_df = query_df(df, repository='amplify-js')
js_df = df
clean_template = lambda text : re.sub(pat, '', text)
js_df[corpus_col] = js_df['title'] + ' ' + js_df['body'].apply(clean_template)

print('Took', time() - start, 'seconds')

Preprocessing data...
Took 302.414612531662 seconds


In [11]:
# Train model
small_df = df
corpus = small_df[corpus_col]
labels = small_df[label_col]

pipe = get_fitted_model(corpus, labels, lemmatize='custom')

Training model...
Took 158.27396488189697 seconds


In [12]:
# Model stats
vocab = pipe['tfidfvectorizer'].get_feature_names()
print('Number of vocab words:', len(vocab))
repo_list = list(set(df['repository']))
print('Available repos:', repo_list)
small_words = [word for word in vocab if len(word) < 3]
print('Small words in vocab:\n', small_words)

Number of vocab words: 65345
Available repos: ['amplify-codegen', 'amplify-ci-support', 'amplify-ios', 'amplify-console', 'aws-sdk-ios', 'amplify-adminui', 'docs', 'amplify-js', 'amplify-js-samples', 'amplify-cli', 'aws-sdk-android', 'amplify-android', 'amplify-flutter']
Small words in vocab:
 ['ad', 'al', 'an', 'as', 'at', 'az', 'bk', 'ca', 'cc', 'cd', 'ce', 'cf', 'ci', 'cl', 'cm', 'co', 'cs', 'ct', 'cu', 'cv', 'db', 'dd', 'de', 'dm', 'do', 'dy', 'eb', 'ec', 'ed', 'em', 'er', 'es', 'fo', 'gb', 'gc', 'gi', 'gm', 'go', 'ha', 'hl', 'ho', 'hr', 'i', 'ic', 'id', 'il', 'in', 'io', 'ip', 'iv', 'j', 'km', 'kt', 'kv', 'la', 'le', 'lf', 'lh', 'li', 'lm', 'm1', 'mb', 'mi', 'nd', 'ni', 'nt', 'nv', 'ob', 'or', 'os', 'pa', 'pi', 'po', 'pr', 'r', 're', 'rh', 's', 'sc', 'si', 'sl', 'sm', 'sn', 'so', 'ss', 'te', 'tl', 'tn', 'tt', 'us', 'vi', 'w', 'wa', 'x']


In [13]:
# Train SM estimator
train_data = 's3://githubmachinelearningstack-rawdatabucket79e6ae92-dvgbsz21ce9v/data'
sklearn_estimator = SKLearn('train_tfidf.py',
                            role=sagemaker.get_execution_role(),
                            instance_type='ml.m5.4xlarge',
                            framework_version='0.23-1',
                            hyperparameters = {'n-best': 10})
sklearn_estimator.fit({'train': train_data})

FileNotFoundError: [Errno 2] No such file or directory: 'train_tfidf.py'

In [ ]:
# Deploy SM predictor
predictor = sklearn_estimator.deploy(instance_type='ml.m5.4xlarge',
                                     initial_instance_count=1)
data = ['password managers autofill']
response = predictor.predict(data)
print(response)

In [ ]:
# with open('extra_tokens.txt', 'w') as f:
#     f.write(str(set(vocab2) - set(vocab)))

In [ ]:
js_issue = ['Password managers autofill']
pipe.predict(js_issue)

In [ ]:
js_issue = ['image file upload fail file size 5 mb']
pipe.predict(js_issue)

In [ ]:
js_issue = ['user endpoint disappear pinpoint']
pipe.predict(js_issue)

In [ ]:
js_issue = query_df(js_df, number=8108)[corpus_col].item()
print(js_issue)
js_issue_ast = BeautifulSoup(js_issue, 'html.parser')
print(js_issue_ast)